In [ ]:
import torch
import torch.nn as nn

In [ ]:
import numpy as np
import random
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import copy
import gym
import retro
from tqdm import tqdm
import collections

In [ ]:
class ImageToPyTorch(gym.ObservationWrapper):
    def __init__(self, env):
        super(ImageToPyTorch, self).__init__(env)
        old_shape = self.observation_space.shape
        self.observation_space = gym.spaces.Box(low=0.0, high=1.0, shape=(old_shape[-1], old_shape[0], old_shape[1]), dtype=np.float32)

    def observation(self, observation):
        return np.moveaxis(observation, 2, 0)

class ScaledFloatFrame(gym.ObservationWrapper):
    def observation(self, obs):
        return np.array(obs).astype(np.float32) / 255.0
    
env = retro.make(game='AirStriker-Genesis')
env = ImageToPyTorch(env)
env = ScaledFloatFrame(env)

In [161]:
class DQN(nn.Module):
    def __init__(self, input_shape, n_actions):
        super(DQN, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(input_shape[0], 32, kernel_size=8, stride=4),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1),
            nn.ReLU()
        )

        conv_out_size = self._get_conv_out(input_shape)
        self.fc = nn.Sequential(
            nn.Linear(conv_out_size, 512),
            nn.ReLU(),
            nn.Linear(512, n_actions)
        )
        
    def _get_conv_out(self, shape):
        o = self.conv(torch.zeros(1, *shape))
        return int(np.prod(o.size()))

    def forward(self, x):
        conv_out = self.conv(x)
        conv_out = conv_out.view(x.size()[0], -1)
        return self.fc(conv_out)

In [162]:
gpu = False

In [163]:
device = torch.device("cuda" if gpu else "cpu")
Q = DQN(env.observation_space.shape, env.action_space.n)

RuntimeError: Calculated padded input size per channel: (3 x 224). Kernel size: (8 x 8). Kernel size can't be greater than actual input size at c:\a\w\1\s\tmp_conda_3.6_091443\conda\conda-bld\pytorch_1544087948354\work\aten\src\thnn\generic/SpatialConvolutionMM.c:50

In [98]:
env.observation_space.shape

(3, 224, 320)

In [99]:
Q.to(device)

DQN(
  (conv): Sequential(
    (0): Conv2d(3, 32, kernel_size=(8, 8), stride=(4, 4))
    (1): ReLU()
    (2): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2))
    (3): ReLU()
    (4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
    (5): ReLU()
  )
  (fc): Sequential(
    (0): Linear(in_features=55296, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=12, bias=True)
  )
)

In [108]:
obs = env.reset()
obs = np.array([env.reset() for _ in range(10)])

In [109]:
Q(torch.FloatTensor(obs).to(device)).shape

torch.Size([10, 12])

In [155]:
def main(nEpisode=10, gamma=0.99, epsilon0=1, epsilonF=0.02, decayingRate=10**(-5), storeQ=1000,
         maxIter=200000, batchSize = 32, replaySize = 10000, replayStartSize=10000, learningRate=1e-4, gpu=False):
    
    device = torch.device("cuda" if gpu else "cpu")
    Q = DQN(env.observation_space.shape, env.action_space.n).to(device)
    QHat = DQN(env.observation_space.shape, env.action_space.n).to(device)
    epsilon = epsilon0
    buffer = collections.deque(maxlen=replaySize)
    loss_fn = torch.nn.MSELoss()
    optimizer = torch.optim.Adam(Q.parameters(), lr=learningRate)
    
    for step in tqdm(range(nEpisode)):
        obs = env.reset()
        for _ in tqdm(range(maxIter)):
            epsilon = max(epsilonF, (1 - decayingRate) * epsilon)
            if np.random.random() < epsilon:
                action = env.action_space.sample()
            else:
                obs = np.array([obs], copy=False)
                obs = torch.tensor(obs).to(device)
                qVals = Q(obs)
                _, action = torch.max(qVals, dim=1)    
                action = int(action.item())
            
            obsNext, reward, done , _ = env.step(action)
            buffer.append(collections.deque([obs, action, reward, done, obsNext]))
            obs = obsNext
            
            if len(buffer) >= replayStartSize:
                indices = np.random.choice(len(buffer), batch_size, replace=False)
                observations, actions, rewards, dones, observationsNext = zip(*[self.buffer[idx] for idx in indices])
                observations, actions, rewards, dones, observationsNext = np.array(states), np.array(actions), np.array(rewards, dtype=np.float32), np.array(dones, dtype=np.uint8), np.array(next_states) 
                minibatch = random.sample(buffer, min(len(buffer), batchSize))
                observationsV = torch.tensor(observations).to(device)
                observationsNextV = torch.tensor(observationsNext).to(device)
                actionsV = torch.tensor(actions).to(device)
                rewardsV = torch.tensor(rewards).to(device)
                doneMask = torch.ByteTensor(dones).to(device)

                stateActionValues = Q(observationsV).gather(1, actionsV.unsqueeze(-1)).squeeze(-1)
                nextStateValues = QHat(observationsNextV).max(1)[0]
                nextStateValues[doneMask] = 0.0
                nextStateValues = nextStateValues.detach()

                expectedStateActionValues = nextStateValues * gamma + rewardsV
                optimizer.zero_grad()
                loss = loss_fn(stateActionValues, expectedStateActionValues)
                loss.backward()
                optimizer.step()
            
            if step % storeQ == 0:
                QHat = copy.deepcopy(Q)
                
            if done:
                break
    return Q

In [156]:
Q = main(20)

RuntimeError: Calculated padded input size per channel: (3 x 224). Kernel size: (8 x 8). Kernel size can't be greater than actual input size at c:\a\w\1\s\tmp_conda_3.6_091443\conda\conda-bld\pytorch_1544087948354\work\aten\src\thnn\generic/SpatialConvolutionMM.c:50

In [144]:
def transform(Q, s, a):
    return float(Q(torch.FloatTensor([s, a])))

In [145]:
q_table = np.ones((env.observation_space.n, env.action_space.n))

for i in range(env.observation_space.n):
    for j in range(env.action_space.n):
        q_table[i, j] = transform(Q, i, j)

In [146]:
def testPolicy (q_table, nEpisode = 2000):
    success = 0
    for _ in range(nEpisode):
        t = 0
        observation = env.reset()
        done  = False
        actionTable = np.argmax(q_table, axis = 1)
        while not done and t < 200:
            action = actionTable[observation]
            observation, reward, done, info = env.step(action)
            t += 1

        if reward == 1:
            success += 1
    return success / nEpisode

In [147]:
testPolicy(q_table)

0.0

In [148]:
q_table

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])